In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    read_loinc_df,
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    read_full_daily_counts_df,
    get_visualization_subtitle,
    apply_theme,
)
from web import for_website

alt.data_transformers.disable_max_rows() # Allow using rows more than 5000

In [ ]:
DATA_RELEASE = "2020-06-15"

In [ ]:
df = read_full_daily_counts_df()

In [ ]:
df

In [ ]:
df.head()

In [ ]:
df = df.replace(-99, np.nan)

plot = alt.Chart(df).mark_line().encode(
    x=alt.X("calendar_date", axis=alt.Axis(title="Date")),
    y=alt.Y("num_patients_in_hospital_on_this_date:Q", axis=alt.Axis(title="Number of Patients in Hospital")),
    color=alt.Color("siteid:N", legend=alt.Legend(title="Site"))
).properties(title={
    "text": ["Number of Hospitalized Patients by Site"], 
    "dx": 50,
    "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE),
    "subtitleColor": "gray",
    "anchor": "middle",
})

plot = apply_theme(plot)

plot

In [ ]:
df["num_sites"] = 1
country_sum_df = df.groupby(["country", "calendar_date"]).sum().reset_index()

In [ ]:
country_sum_df

In [ ]:
top_plot = alt.Chart(country_sum_df).mark_line().encode(
    x=alt.X("calendar_date", axis=alt.Axis(title=None)),
    y=alt.Y("num_patients_in_hospital_on_this_date:Q", axis=alt.Axis(title="Number of Patients in Hospital")),
    color=alt.Color("country:N", legend=alt.Legend(title="Country"))
)

bottom_plot = alt.Chart(country_sum_df).mark_bar().encode(
    x=alt.X("calendar_date", axis=alt.Axis(title="Date")),
    y=alt.Y("num_sites:Q", axis=alt.Axis(title="# of sites")),
    color=alt.Color("country:N", legend=alt.Legend(title="Country"))
).properties(height=80)

plot = alt.vconcat(top_plot, bottom_plot).resolve_scale(x="independent", color="shared").properties(title={
    "text": ["Number of Hospitalized Patients by Country"], 
    "dx": 50,
    "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE),
    "subtitleColor": "gray",
    "anchor": "middle",
})

plot = apply_theme(plot)

plot